In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA as sklearnPCA
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from pylab import rcParams    
rcParams['figure.dpi']=300 

In [ ]:
os.chdir(r"C:\Users\fanyu\Desktop\Jupiter")
data = pd.read_csv("all_news_cleaned.csv",encoding = "latin1")

In [ ]:
import gensim.models
model = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [ ]:
text_column = data['cleaned_content']

text_data = []
for text in text_column:
    sentence_list = nltk.sent_tokenize(text) 
    text_data.extend(sentence_list)
    
preprocessed_data = []
for sentence in text_data:
    preprocessed_sentence = [word.lower() for word in sentence.split() if word.isalpha()] 
    preprocessed_data.append(preprocessed_sentence)

embedding_data = []
for sentence in preprocessed_data:
    sentence_embedding = [model.get_vector(word) for word in sentence if word in model.key_to_index]
    if sentence_embedding:
        embedding_data.append(sum(sentence_embedding) / len(sentence_embedding))
    else:
        embedding_data.append(None)

np.array(embedding_data).shape

embedding = pd.DataFrame(embedding_data, columns=['Dim{}'.format(i) for i in range(1, np.array(embedding_data).shape[1]+ 1)])

from sklearn.decomposition import PCA as sklearnPCA
pca = sklearnPCA(n_components=2)
pca.fit_transform(embedding)

data3=pd.read_csv('all_news_cleaned.csv',index_col=0)
data3[['Dim1','Dim2']]=pca.fit_transform(embedding) 

sns.scatterplot(data3,x="Dim1",y="Dim2",hue="source")
plt.title("Word Embedding (Semantic Relationship)")
plt.savefig('Word_embedding.png', bbox_inches='tight')